# QM9 Pro.
Not sure what I’m supposed to call this file.  
Plan is to keep this repo much neater than before.  
Getting to work now.

In [8]:
import torch_geometric
import torch
from torch_geometric.datasets import QM9
from torch_geometric.data import DataLoader

In [12]:
torch.manual_seed(2002)
dataset = QM9(root='QM9/')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

/usr/local/Caskroom/miniconda/base/envs/GDL/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
print(next(iter(dataloader)))

DataBatch(x=[567, 11], edge_index=[2, 1182], edge_attr=[1182, 4], y=[32, 19], pos=[567, 3], idx=[32], name=[32], z=[567], batch=[567], ptr=[33])
